In [1]:
# import necessary modules
import pandas as pd
import time as time
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import warnings
warnings.filterwarnings("ignore")

In [2]:
# connect to Cassandra
server_config = {
                 "host": "127.0.0.1",
                 "port": "9042",
                 "user": "",
                 "password": "",
                 "keySpace": "cds502"
     }


keyspace = server_config['keySpace']

auth_provider = PlainTextAuthProvider(username=server_config['user'],password=server_config['password'])

node_ips = [server_config['host']]



cluster = Cluster(contact_points=node_ips, load_balancing_policy=None, port=int(server_config['port']), auth_provider=auth_provider, protocol_version=3)
session = cluster.connect()
session.default_timeout = 30000000
session.set_keyspace(keyspace)

### Number of arrested crimes by year

In [3]:
# first query
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session = cluster.connect()
session.default_timeout = 30000000
session.row_factory = pandas_factory
session.default_fetch_size = None
session.set_keyspace(keyspace)

start_time = time.time()
query = session.execute("""SELECT "Year", COUNT("Case Number") FROM arrests WHERE "Arrest"=True GROUP BY "Year" ALLOW FILTERING;""")
query_time1 = (time.time()-start_time)
df = query._current_rows
print(f"Time to execute query: {query_time1:.2f}s")

Time to execute query: 20.82s


In [4]:
df = df.sort_values(by=["Year"])
df = df.reset_index(drop=True)

for index, row in df.iterrows():
    print(f"Number of crimes in year {row['Year']}: {row['system.count(Case Number)']}")

Number of crimes in year 2001: 141900
Number of crimes in year 2002: 141542
Number of crimes in year 2003: 141568
Number of crimes in year 2004: 144681
Number of crimes in year 2005: 140890
Number of crimes in year 2006: 135376
Number of crimes in year 2007: 131838
Number of crimes in year 2008: 109940
Number of crimes in year 2009: 110744
Number of crimes in year 2010: 100424
Number of crimes in year 2011: 96181
Number of crimes in year 2012: 90539
Number of crimes in year 2013: 86419
Number of crimes in year 2014: 79258
Number of crimes in year 2015: 69650
Number of crimes in year 2016: 52444
Number of crimes in year 2017: 38517


In [5]:
df

,Year,system.count(Case Number)
0,2001,141900
1,2002,141542
2,2003,141568
3,2004,144681
4,2005,140890
5,2006,135376
6,2007,131838
7,2008,109940
8,2009,110744
9,2010,100424


### Number of crimes by location

In [6]:
# second query

session = cluster.connect()
session.default_timeout = 30000000
session.row_factory = pandas_factory
session.default_fetch_size = None
session.set_keyspace(keyspace)

start_time = time.time()
query2=session.execute("""SELECT "Location Description", COUNT("Case Number") FROM crime_by_location GROUP BY "Location Description";""")
query2_time = time.time()-start_time
df2 = query2._current_rows
print(f"Time to execute query: {query2_time:.2f}s")

Time to execute query: 32.56s


In [7]:
df2_sort = df2.sort_values(by=["system.count(Case Number)"], ascending=False)
df2_sort = df2_sort.reset_index(drop=True)
df2_sort.columns = ["Location", "Number of Crimes"]

print("Top 5 Locations with Highest Number of Crimes: ")
df2_top5 = df2_sort.iloc[:5, :]
print(df2_top5)

Top 5 Locations with Highest Number of Crimes: 
    Location  Number of Crimes
0     STREET           1704536
1  RESIDENCE           1091375
2  APARTMENT            658789
3   SIDEWALK            641954
4      OTHER            243977


In [8]:
for index, row in df2.iterrows():
    print(f"Number of crimes in {row['Location Description']}: {row['system.count(Case Number)']}")

Number of crimes in : 2799
Number of crimes in WAREHOUSE: 8878
Number of crimes in CLEANERS/LAUNDROMAT: 1
Number of crimes in CTA GARAGE / OTHER PROPERTY: 9630
Number of crimes in STREET: 1704536
Number of crimes in RETAIL STORE: 68
Number of crimes in PORCH: 269
Number of crimes in LOADING DOCK: 1
Number of crimes in AIRPORT PARKING LOT: 573
Number of crimes in NEWSSTAND: 224
Number of crimes in AIRCRAFT: 528
Number of crimes in DRIVEWAY: 15
Number of crimes in CTA "L"" PLATFORM": 4
Number of crimes in LAGOON: 1
Number of crimes in GOVERNMENT BUILDING: 1
Number of crimes in CONSTRUCTION SITE: 12437
Number of crimes in BANQUET HALL: 1
Number of crimes in CTA TRACKS - RIGHT OF WAY: 66
Number of crimes in CLEANING STORE: 4634
Number of crimes in MOTEL: 5
Number of crimes in BAR OR TAVERN: 33549
Number of crimes in PUBLIC HIGH SCHOOL: 2
Number of crimes in BARBER SHOP/BEAUTY SALON: 12
Number of crimes in OTHER: 243977
Number of crimes in GAS STATION DRIVE/PROP.: 43
Number of crimes in CHA

In [9]:
df2.head(10)

,Location Description,system.count(Case Number)
0,,2799
1,WAREHOUSE,8878
2,CLEANERS/LAUNDROMAT,1
3,CTA GARAGE / OTHER PROPERTY,9630
4,STREET,1704536
5,RETAIL STORE,68
6,PORCH,269
7,LOADING DOCK,1
8,AIRPORT PARKING LOT,573
9,NEWSSTAND,224


### Number of robbery cases in District 1

In [23]:
cluster = Cluster(contact_points=node_ips, load_balancing_policy=None, port=int(server_config['port']), auth_provider=auth_provider, protocol_version=3)
session = cluster.connect()
session.default_timeout = 30000000
session.row_factory = pandas_factory
session.default_fetch_size = None
session.set_keyspace(keyspace)

start_time = time.time()
query3=session.execute("""SELECT count(*) FROM crime WHERE "District"=1 AND "Primary Type"='ROBBERY' ALLOW FILTERING;""", timeout=None)
query3_time = time.time()-start_time
df3 = query3._current_rows
print(f"Time to execute query: {query3_time:.2f}s")

Time to execute query: 54.74s


In [11]:
print(f"Number of robbery cases in District 1: {df3.iloc[0,0]}")

Number of robbery cases in District 1: 6031


In [12]:
df3

,count
0,6031


### Number of theft crime by location in year 2015

In [13]:
# fourth query

session = cluster.connect()
session.default_timeout = 30000000
session.row_factory = pandas_factory
session.default_fetch_size = None
session.set_keyspace(keyspace)

start_time = time.time()
query4=session.execute("""SELECT "Location Description", COUNT("Case Number") FROM crime_by_location WHERE "Year"=2015 AND "Primary Type"='THEFT' GROUP BY "Location Description" ALLOW FILTERING;""")
query4_time = time.time()-start_time
df4 = query4._current_rows
print(f"Time to execute query: {query4_time:.2f}s")

Time to execute query: 16.52s


In [14]:
df4_sort = df4.sort_values(by=["system.count(Case Number)"], ascending=False)
df4_sort = df4_sort.reset_index(drop=True)
df4_sort.columns = ["Location", "Number of Theft"]

print("Top 5 Locations with Highest Number of Thefts: ")
df4_top5 = df4_sort.iloc[:5, :]
print(df4_top5)

Top 5 Locations with Highest Number of Thefts: 
           Location  Number of Theft
0            STREET            14451
1         RESIDENCE             4087
2         APARTMENT             3673
3  DEPARTMENT STORE             3317
4             OTHER             3296


In [19]:
for index, row in df4.iterrows():
    print(f"Number of theft crimes in year 2015 at {row['Location Description']}: {row['system.count(Case Number)']}")

Number of theft crimes in year 2015 at : 1
Number of theft crimes in year 2015 at WAREHOUSE: 89
Number of theft crimes in year 2015 at CTA GARAGE / OTHER PROPERTY: 97
Number of theft crimes in year 2015 at STREET: 14451
Number of theft crimes in year 2015 at AIRPORT PARKING LOT: 33
Number of theft crimes in year 2015 at NEWSSTAND: 3
Number of theft crimes in year 2015 at AIRCRAFT: 42
Number of theft crimes in year 2015 at CONSTRUCTION SITE: 144
Number of theft crimes in year 2015 at CTA TRACKS - RIGHT OF WAY: 3
Number of theft crimes in year 2015 at CLEANING STORE: 20
Number of theft crimes in year 2015 at BAR OR TAVERN: 782
Number of theft crimes in year 2015 at OTHER: 3296
Number of theft crimes in year 2015 at BRIDGE: 1
Number of theft crimes in year 2015 at PAWN SHOP: 25
Number of theft crimes in year 2015 at DRUG STORE: 622
Number of theft crimes in year 2015 at HOSPITAL BUILDING/GROUNDS: 285
Number of theft crimes in year 2015 at POOL ROOM: 9
Number of theft crimes in year 2015 a

In [15]:
df4

,Location Description,system.count(Case Number)
0,,1
1,WAREHOUSE,89
2,CTA GARAGE / OTHER PROPERTY,97
3,STREET,14451
4,AIRPORT PARKING LOT,33
...,...,...
96,FOREST PRESERVE,6
97,AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA,46
98,"SCHOOL, PUBLIC, BUILDING",876
99,AIRPORT TERMINAL UPPER LEVEL - SECURE AREA,95


### Number of thefts occuring in residences with arrests being made in year 2010

In [24]:
session = cluster.connect()
session.default_timeout = 30000000
session.row_factory = pandas_factory
session.default_fetch_size = None
session.set_keyspace(keyspace)

start_time = time.time()
query5=session.execute("""SELECT count(*) FROM crime WHERE "Primary Type"='THEFT' and "Location Description"='RESIDENCE' and "Arrest"= True and "Year"=2010 ALLOW FILTERING;""")
query5_time = time.time()-start_time
df5 = query5._current_rows
print(f"Time to execute query: {query5_time:.2f}s")

Time to execute query: 54.95s


In [17]:
print(f"Number of robbery cases in District 1: {df5.iloc[0,0]}")

Number of robbery cases in District 1: 154


In [18]:
df5

,count
0,154
